In [135]:
%reload_ext autoreload
%autoreload 3

%matplotlib qt
import matplotlib.pyplot as plt                                                                                         # Import matplotlib.pyplot

import pandas
import tkinter as tk
from tkinter import filedialog
import numpy
import re
import os
import time
import glob
from shutil import rmtree
import random  
import warnings
warnings.filterwarnings('ignore')

import chrono_reader as chr
import chrono_plotter as chp
import chrono_rithm as chrt
import chrono_simulation as chs


### SELECT THE FILE TO SAVE THE SIMULATIONS

In [3]:
root = tk.Tk()                                                                                                          # Create the root window
root.attributes('-topmost',True)                                                                                        # Keep the window on top of others
root.iconify()                                                                                                          # Hide the root window

main_save_folder = filedialog.askdirectory(title="Select the folder to save the simulated data", parent=root)           # Ask for the folder to save the simulated data
root.destroy()                                                                                                          # Destroy the root window

print(main_save_folder)                                                                                                 # Print the folder to save the simulated data

E:/github/circadipy/src/simulated_data


### RUN AND SAVE THE SIMULATIONS

Each simulation has X cycles (num_cycles), each cycle can be configured with a single activity period. Example: for simulations with a light cycle of 12 hours light and 12 hours dark for 5 days, you have 5 days of this cycle (cycle_days) with a period (activity_period) of 24 hours. 

In [136]:
num_cycles = 2                                                                                                          # Number of cycles in each simulation
simulations_num = 10                                                                                                    # Number of simulations to run
sampling_frequency = '30T'                                                                                              # Sampling frequency of the simulated data (30T = 30 minutes)
# Set seed
random.seed(42)

types = ['sine', 'square', 'triangle', 'sawtooth']                                                                      # Types of simulated data
types = ['triangle']
noise_levels = [0.1, 0.5, 1, 5, 10]

simulations = {}                                                                                                        # Dictionary to store the simulations data
for t in types:    
    simulations[t] = {}                                                                                                 # Create a dictionary to store the type data                                                                                       # Create a dictionary to store the noise level data
    for s in range(simulations_num):                                                                                    # For each simulation
        s = str(s)
        simulations[t][s] = {}                                                                                          # Create a dictionary to store the simulation data
        simulations[t][s]['cycles_days'] = [random.randint(3, 10) for _ in range(num_cycles)]                           # Randomly generate the number of days in each cycle (5 cycles)
        simulations[t][s]['activity_period'] = [random.randint(2200, 2600)/100 for _ in range(num_cycles)]              # Randomly generate the activity period in each cycle (5 cycles)

In [138]:
for t in types:    
    for s in simulations[t]:                                                                                            # For each simulation
        file_name = 'simulated_data_' + s + '_' + t                                                                     # Create the file name for the simulated data without noise
        cycle_days = simulations[t][s]['cycles_days']                                                                   # Get the number of days in each cycle
        activity_period = simulations[t][s]['activity_period']                                                          # Get the activity period in each cycle

        data_raw_without_noise = chs.simulate_protocol(file_name, main_save_folder, sampling_frequency, 
                                                       cycle_days, activity_period, signal_type = t, 
                                                       noise = False, snr_db = 20, only_positive = True, 
                                                       remove_file = True)                                              # Simulate the data with noise (chrono_simulation.py)
        simulations[t][s][file_name] = data_raw_without_noise                                                           # Store the data with noise in the simulations dictionary
        
        for n in noise_levels:       
            file_name_noise = 'simulated_data_' + s + '_' + t + '_' + str(n)                                            # Create the file name for the simulated data with noise
            data_raw_with_noise = chs.simulate_protocol(file_name_noise, main_save_folder, sampling_frequency, 
                                                        cycle_days, activity_period, signal_type = t, 
                                                        noise = True, snr_db = n, only_positive = True, 
                                                        remove_file = True)                                             # Simulate the data with noise (chrono_simulation.py)

        
            simulations[t][s][file_name_noise] = data_raw_with_noise                                                    # Store the data without noise in the simulations dictionary

        simulation_file = open(main_save_folder + '/simulation_' + s + '_' + t + '_description.txt', 'w')               # Create a file to store the simulation description to be used later
        simulation_file.write('Simulation number: ' + s + '\n')                                                         # Write the simulation number
        simulation_file.write('Cycles days: ' + str(simulations[t][s]['cycles_days']) + '\n')                              # Write the cycles days
        simulation_file.write('Activity period: ' + str(simulations[t][s]['activity_period']) + '\n')                      # Write the activity period
        simulation_file.close()                                                                                         # Close the file

        print('Simulation ' + s + ' done!')                                                                             # Print a message to indicate that the simulation is done

Saved successfully in E:/github/circadipy/src/simulated_data/simulated_data_0_triangle.asc
Saved successfully in E:/github/circadipy/src/simulated_data/simulated_data_0_triangle_0.1.asc
Saved successfully in E:/github/circadipy/src/simulated_data/simulated_data_0_triangle_0.5.asc
Saved successfully in E:/github/circadipy/src/simulated_data/simulated_data_0_triangle_1.asc
Saved successfully in E:/github/circadipy/src/simulated_data/simulated_data_0_triangle_5.asc
Saved successfully in E:/github/circadipy/src/simulated_data/simulated_data_0_triangle_10.asc
Simulation 0 done!
Saved successfully in E:/github/circadipy/src/simulated_data/simulated_data_1_triangle.asc
Saved successfully in E:/github/circadipy/src/simulated_data/simulated_data_1_triangle_0.1.asc
Saved successfully in E:/github/circadipy/src/simulated_data/simulated_data_1_triangle_0.5.asc
Saved successfully in E:/github/circadipy/src/simulated_data/simulated_data_1_triangle_1.asc
Saved successfully in E:/github/circadipy/src/

In [128]:
fig = plt.figure(figsize = (12, 5))                                                                                     # Create a figure
ax = fig.add_subplot(111)                                                                                               # Add a subplot
ax.plot(data_raw_with_noise, color = 'maroon', linewidth = 2)                                                           # Plot the data with noise
ax.plot(data_raw_without_noise, color = 'dimgray', linewidth = 2)                                                       # Plot the data without noise
ax.set_xlabel('Time (Hours)')                                                                                           # Set the x label
ax.set_ylabel('Activity')                                                                                               # Set the y label
ax.legend(['With noise', 'Without noise'])                                                                              # Set the legend
plt.show()                                                                                                              # Show the plot

### IMPORT THE SIMULATIONS AND BUILD THE PROTOCOLS TO BE STUDIED

In [139]:
individual_files = glob.glob(main_save_folder + "\\*.asc")                                                              # Get the files for each simulation (with and without noise) 

In [140]:
zt_0_time = 0                                                                                                           # Set the time for the first ZT (0)

simulations = {}                                                                                                        # Create a dictionary to store the simulations data
for file in individual_files:                                                                                           # For each file (each simulation)
    expression = r"\\simulated_data_(.*?)\_(.*?)\.asc"                                                                  # Create a regular expression to get the simulation number and if the data has noise or not
    name_file = re.search(expression, file)                                                                             # Search for the regular expression in the file name
    number = name_file.group(1)                                                                                         # Get the simulation number
    sig_noise = name_file.group(2)                                                                                      # Get if the data has noise or not
    
    if '_' in sig_noise:                                                                                                # If the data has noise
        sig = sig_noise.split('_')[0]                                                                                   # Get only the noise value
        noise = sig_noise.split('_')[1]                                                                                 # Get only the noise value
    else:                                                                                                               # If the data does not have noise
        sig = sig_noise                                                                                                 # Get the value
        noise = '0'                                                                                                     # Set the noise value to 0

    name = number + '_' + sig + '_' + noise                                                                             # Create the name of the simulation

    if sig not in simulations:                                                                                          # If the noise value is not in the dictionary
        simulations[sig] = {}
        simulations[sig][noise] = {}
    else:
        if noise not in simulations[sig]:
            simulations[sig][noise] = {}
    
    simulations[sig][noise][name] = {}                                                                                  # Create a dictionary to store each simulation data (each simulation)
    simulations[sig][noise][name]['file'] = name_file                                                                   # Store the file name
    simulation_file = main_save_folder + '/simulation_' + number + '_' + sig + '_description.txt'                       # Store the simulation description file name
    simulations[sig][noise][name]['simulation_file'] =simulation_file                                                   # Store the simulation description file name

    simulation_file = open(simulations[sig][noise][name]['simulation_file'], 'r')                                       # Open the simulation description file
    lines = simulation_file.readlines()                                                                                 # Read the lines
    simulation_file.close()                                                                                             # Close the file

    cycle_experssion = r"Cycles days: \[(.*?)\]"                                                                        # Create a regular expression to get the number of days in each cycle
    activity_expression = r"Activity period: \[(.*?)\]"                                                                 # Create a regular expression to get the activity period in each cycle
    cycle_days = re.search(cycle_experssion, lines[1]).group(1)                                                         # Get the number of days in each cycle
    activity_period = re.search(activity_expression, lines[2]).group(1)                                                 # Get the activity period in each cycle

    simulations[sig][noise][name]['cycle_days'] = [int(x) for x in cycle_days.split(', ')]                              # Store the number of days in each cycle
    simulations[sig][noise][name]['activity_period'] = [float(x) for x in activity_period.split(', ')]                  # Store the activity period in each cycle

    labels_dict = {'cycle_types': ['DD', 'DD'], 
                   'test_labels': ['1','2'], 
                   'cycle_days': simulations[sig][noise][name]['cycle_days']}                                           # Create a dictionary to store to pass as argument to the read_protocol function

    print(file)

    protocol = chr.read_protocol(name, file, zt_0_time = zt_0_time, labels_dict = labels_dict, 
                                 type = 'generic', consider_first_day = True)                                           # Read the protocol (chrono_reader.py)
    # protocol.resample('30T', method = 'sum')                                                                          # Resample the data (chrono_reader.py)
    # protocol.apply_filter(window = 3, type = 'moving_average', order = 2, reverse = False)                            # Apply a filter to the data (chrono_reader.py)
    # protocol.normalize_data(type = 'minmax', per_day = True)                                                          # Normalize the data (chrono_reader.py)

    simulations[sig][noise][name]['protocol'] = protocol                                                                # Store the protocol in the simulations dictionary

print(simulations)

E:/github/circadipy/src/simulated_data\simulated_data_0_sawtooth.asc
E:/github/circadipy/src/simulated_data\simulated_data_0_sawtooth_0.1.asc
E:/github/circadipy/src/simulated_data\simulated_data_0_sawtooth_0.5.asc
E:/github/circadipy/src/simulated_data\simulated_data_0_sawtooth_1.asc
E:/github/circadipy/src/simulated_data\simulated_data_0_sawtooth_10.asc
E:/github/circadipy/src/simulated_data\simulated_data_0_sawtooth_5.asc
E:/github/circadipy/src/simulated_data\simulated_data_0_sine.asc
E:/github/circadipy/src/simulated_data\simulated_data_0_sine_0.1.asc
E:/github/circadipy/src/simulated_data\simulated_data_0_sine_0.5.asc
E:/github/circadipy/src/simulated_data\simulated_data_0_sine_1.asc
E:/github/circadipy/src/simulated_data\simulated_data_0_sine_10.asc
E:/github/circadipy/src/simulated_data\simulated_data_0_sine_5.asc
E:/github/circadipy/src/simulated_data\simulated_data_0_square.asc
E:/github/circadipy/src/simulated_data\simulated_data_0_squarelowpass.asc
E:/github/circadipy/src/s

In [142]:
#Select only 'triangle' in simulations dictionary
simulations = {k: v for k, v in simulations.items() if k == 'triangle'}
print(simulations)

{'triangle': {'0': {'0_triangle_0': {'file': <re.Match object; span=(38, 68), match='\\simulated_data_0_triangle.asc'>, 'simulation_file': 'E:/github/circadipy/src/simulated_data/simulation_0_triangle_description.txt', 'cycle_days': [4, 3], 'activity_period': [25.79, 23.4], 'protocol': <chrono_reader.read_protocol object at 0x000001EA1A9EADF0>}, '1_triangle_0': {'file': <re.Match object; span=(38, 68), match='\\simulated_data_1_triangle.asc'>, 'simulation_file': 'E:/github/circadipy/src/simulated_data/simulation_1_triangle_description.txt', 'cycle_days': [6, 6], 'activity_period': [22.71, 25.77], 'protocol': <chrono_reader.read_protocol object at 0x000001EA0562B490>}, '2_triangle_0': {'file': <re.Match object; span=(38, 68), match='\\simulated_data_2_triangle.asc'>, 'simulation_file': 'E:/github/circadipy/src/simulated_data/simulation_2_triangle_description.txt', 'cycle_days': [4, 4], 'activity_period': [25.02, 24.16], 'protocol': <chrono_reader.read_protocol object at 0x000001EA71EEF3

### CONFIGURE THE FOLDERS TO RECEIVE THE ANALYSIS RESULTS

In [143]:
remove_file = False                                                                                                     # Set if the file should be removed or not if it already exists
parent_path = os.path.abspath(os.path.join(main_save_folder, os.pardir))                                                # Get the parent folder
results_folder = parent_path + "\\results"                                                                              # Set the results folder
print(results_folder)                                                                                                   # Print the results folder

def reset_folder(folder):                                                                                               # Function to reset the folder
    for root, dirs, files in os.walk(folder):                                                                           # For each file in the folder
        for f in files:                                                                                                 # For each file
            os.unlink(os.path.join(root, f))                                                                            # Remove the file
        for d in dirs:                                                                                                  # For each folder
            rmtree(os.path.join(root, d))                                                                               # Remove the folder

check_folder = os.path.isdir(results_folder)                                                                            # Check if the folder exists
if not check_folder:                                                                                                    # If the folder doesn't exist
    os.makedirs(results_folder)                                                                                         # Create the folder to save the file
    print("Results folder created")                                                                                     # Print that the folder was created
    for count, key in enumerate(list(simulations.keys())):                                                              # For each simulation
        for key_2 in list(simulations[key].keys()):                                                                     # For each key in the simulation   
            simulations[key][key_2]['save_folder'] = results_folder + '\\' + key + '_' + key_2                          # Set the folder to save the file
            check_simulation_folder = os.path.isdir(simulations[key][key_2]['save_folder'])                             # Check if the file exists in the folder
            if check_simulation_folder:                                                                                 # If the file exists
                if remove_file:                                                                                         # If remove_file is True
                    reset_folder(results_folder + '\\' + key + '_' + key_2)                                             # Reset the folder
                    print("Folder " + key + '_' + key_2 + " cleaned!")                                                  # Print that the folder was cleaned
                else:                                                                                                   # If remove_file is False
                    raise ValueError("Folder already exists")                                                           # Raise an error
            else:                                                                                                       # If the file doesn't exist
                os.makedirs(simulations[key][key_2]['save_folder'])                                                     # Create the folder to save the file
                print("Folder " + key + '_' + key_2 + " created!")                                                      # Print that the folder was created
else:                                                                                                                   # If the folder exists
    print("Results folder already exists")                                                                              # Print that the folder already exists
    for count, key in enumerate(list(simulations.keys())):                                                              # For each simulation
        for key_2 in list(simulations[key].keys()):                                                                     # For each key in the simulation    
            simulations[key][key_2]['save_folder'] = results_folder + '\\' + key + '_' + key_2                          # Set the folder to save the file
            check_simulation_folder = os.path.isdir(simulations[key][key_2]['save_folder'])                             # Check if the file exists in the folder
            if check_simulation_folder:                                                                                 # If the file exists
                if remove_file:                                                                                         # If remove_file is True
                    reset_folder(results_folder + '\\' + key + '_' + key_2)                                             # Reset the folder
                    print("Folder " + key + '_' + key_2 + " cleaned!")                                                  # Print that the folder was cleaned
                else:                                                                                                   # If remove_file is False
                    raise ValueError("Folder already exists")                                                           # Raise an error
            else:                                                                                                       # If the file doesn't exist
                os.makedirs(simulations[key][key_2]['save_folder'])                                                     # Create the folder to save the file
                print("Folder " + key + '_' + key_2 + " created!")                                                      # Print that the folder was created


E:\github\circadipy\src\results
Results folder already exists
Folder triangle_0 created!
Folder triangle_0.1 created!
Folder triangle_0.5 created!
Folder triangle_1 created!
Folder triangle_10 created!
Folder triangle_5 created!


### RUN THE MODELING

- The "time_shape" can be 'continuous', 'meadian' or 'mean'. If it is 'continuous', all samples will be cosider to 
calculate the model. If it is 'median' or 'mean', the samples of activity will be grouped by mean/median along one day. 
- The "step" is the time step to calculate the model (in hours). 
- The "start/end_time" defines the period that the model will try to fit. E.g. if the start time is 24, the end time 
is 26, and the step is 0.5, the model will try to fit the data to a cosine curve with period equal to 24, 24.5, 25,
25.5 and 26 hours.
- The "n_components" is the number of components that the model will try to fit. If it is a list, the model will try to 
fit all the number of components in the list. E.g. if n_components = [1,2,3], the model will try to fit the data with
1, 2 and 3 components.

In [144]:
dict = {'time_shape': 'continuous',                     
        'step': 0.01, 
        'start_time': 22, 
        'end_time': 26, 
        'n_components': [1]}                                                                                                                            # Create a dictionary to pass as argument to the fit_cosinor function

best_models = []                                                                                                                                        # Create an empty list to store the best models

for count, sig in enumerate(simulations):                                                                                                               # For each simulation (simulation)
    for noise in simulations[sig]:                                                                                                                      # For each noise level
        save_folder = simulations[sig][noise]['save_folder']                                                                                            # Get the folder to save the file

        for sim in simulations[sig][noise]:                                                                                                             # For each simulation         
            if sim != 'save_folder':                                                                                          
                init = time.time()                                                                                                                      # Get the initial time

                best_models, _ = chrt.fit_cosinor(simulations[sig][noise][sim]['protocol'], dict = dict, save_folder = save_folder)                     # Fit the cosinor to the data (chrono_rithm.py)
                best_models_fixed, _ = chrt.fit_cosinor_fixed_period(simulations[sig][noise][sim]['protocol'], best_models, save_folder = save_folder)  # Fix the best period calculated and fit the cosinor for each day using this period (chrono_rithm.py)
                simulations[sig][noise][sim]['best_models'] = best_models                                                                               # Store the best models in the simulations dictionary
                simulations[sig][noise][sim]['best_models_fixed'] = best_models_fixed                                                                   # Store the best models fixed in the simulations dictionary
                
                end = time.time() - init                                                                                                                # Get the time elapsed

                print("Cosinor fitted to " + sig + ' ' + noise + " and results saved!")                                                                 # Print that the cosinor was fitted and the results saved
                print("Time elapsed: " + "{:.2f}".format(end) + " seconds")                                                                             # Print the time elapsed

Cosinor fitted to triangle 0 and results saved!
Time elapsed: 9.47 seconds
Cosinor fitted to triangle 0 and results saved!
Time elapsed: 13.23 seconds
Cosinor fitted to triangle 0 and results saved!
Time elapsed: 11.00 seconds
Cosinor fitted to triangle 0 and results saved!
Time elapsed: 9.71 seconds
Cosinor fitted to triangle 0 and results saved!
Time elapsed: 12.80 seconds
Cosinor fitted to triangle 0 and results saved!
Time elapsed: 14.72 seconds
Cosinor fitted to triangle 0 and results saved!
Time elapsed: 10.68 seconds
Cosinor fitted to triangle 0 and results saved!
Time elapsed: 16.77 seconds
Cosinor fitted to triangle 0 and results saved!
Time elapsed: 15.16 seconds
Cosinor fitted to triangle 0 and results saved!
Time elapsed: 13.42 seconds
Cosinor fitted to triangle 0.1 and results saved!
Time elapsed: 10.40 seconds
Cosinor fitted to triangle 0.1 and results saved!
Time elapsed: 12.57 seconds
Cosinor fitted to triangle 0.1 and results saved!
Time elapsed: 10.40 seconds
Cosinor 

### COMPARE THE MODEL RESULTS FOR EACH CYCLE

In [133]:
read = True                                                                                                                               # If you already ran the analysis, set this to True to read the data instead of running the analysis again
model_error = {}                                                                                                                          # Create an empty list to store the model error

for sig in animals:
      model_error[sig] = {}                                                                                                               # Create an empty list to store the model error for each type of signal
      for noise in animals[sig]:
            model_error[sig][noise] = []                                                                                                  # Create an empty list to store the model error for each amplitude of noise
            for sim in animals[sig][noise]:                                                                                               # For each animal (simulation)
                  if sim != 'save_folder':  
                        if read == True:
                              best_models = pandas.read_csv(save_folder + '/cosinor_' + sim + '_.xlsx')                                   # Read the data
                              animals[sig][noise][sim]['best_models'] = best_models                                                       # Store the data in the dictionary
                              best_models_fixed = pandas.read_csv(save_folder + '/cosinor_per_day_fixed_period_' + sim + '_fixed.xlsx')   # Read the data
                              animals[sig][noise][sim]['best_models_fixed'] = best_models_fixed                                           # Store the data in the dictionary
                        
                        animals[sig][noise][sim]['activity_period_model'] = list(animals[sig][noise][sim]['best_models']['period'])       # Get the activity period calculated by the model
                        animal_model_error = [abs(a - b) for a,b in zip(animals[sig][noise][sim]['activity_period_model'], 
                                                                        animals[sig][noise][sim]['activity_period'])]                     # Calculate the error between the model and the real activity period used to generate the data
                        
                        model_error[sig][noise].extend(animal_model_error)                                                                # Store the model error in the list

            print("Signal: " + sig + ", noise: " + noise)
            print("Mean error: " + "{:.2f}".format(numpy.mean(model_error[sig][noise])) + " +/- " + 
                  "{:.2f}".format(numpy.std(model_error[sig][noise])) + " hours")                                                         # Print the mean and standard deviation of the model error

Signal: sawtooth, noise: 0
Mean error: 0.29 +/- 0.32 hours
Signal: sawtooth, noise: 0.1
Mean error: 0.46 +/- 0.38 hours
Signal: sawtooth, noise: 0.5
Mean error: 0.54 +/- 0.61 hours
Signal: sawtooth, noise: 1
Mean error: 0.44 +/- 0.46 hours
Signal: sawtooth, noise: 10
Mean error: 0.35 +/- 0.36 hours
Signal: sawtooth, noise: 5
Mean error: 0.30 +/- 0.35 hours
Signal: sine, noise: 0
Mean error: 0.00 +/- 0.00 hours
Signal: sine, noise: 0.1
Mean error: 0.29 +/- 0.25 hours
Signal: sine, noise: 0.5
Mean error: 0.27 +/- 0.28 hours
Signal: sine, noise: 1
Mean error: 0.25 +/- 0.21 hours
Signal: sine, noise: 10
Mean error: 0.07 +/- 0.08 hours
Signal: sine, noise: 5
Mean error: 0.12 +/- 0.14 hours
Signal: square, noise: 0
Mean error: 0.06 +/- 0.07 hours
Signal: square, noise: 0.1
Mean error: 0.28 +/- 0.35 hours
Signal: square, noise: 0.5
Mean error: 0.15 +/- 0.13 hours
Signal: square, noise: 1
Mean error: 0.21 +/- 0.30 hours
Signal: square, noise: 10
Mean error: 0.06 +/- 0.08 hours
Signal: square, 

### PLOT THE RESULTS

It may take a while to run

In [145]:
format = 'png'

for sig in animals:
      for noise in animals[sig]:
            for sim in animals[sig][noise]:                                                                                # For each animal (simulation)
                  save_folder = animals[sig][noise]['save_folder']
                  if sim != 'save_folder': 
                        chp.actogram_bar(animals[sig][noise][sim]['protocol'], first_hour = 18, save_folder = save_folder, save_suffix = 'bar', adjust_figure = [2, 0.95, 0.80, 0.20, 0.05], format = format)
                        chp.actogram_colormap(animals[sig][noise][sim]['protocol'], first_hour = 18, save_folder = save_folder, save_suffix = 'colormap', adjust_figure = [2, 0.95, 0.80, 0.20, 0.05], norm_color = None, format = format)
                        chp.data_periodogram(animals[sig][noise][sim]['protocol'], time_shape = 'continuous', method = 'periodogram', max_period = 48, unit_of_measurement = 'AMPLITUDE', save_folder = save_folder, save_suffix = 'periodogram', format = format)
                        chp.data_periodogram(animals[sig][noise][sim]['protocol'], time_shape = 'continuous', method = 'welch', max_period = 48, unit_of_measurement = 'AMPLITUDE', save_folder = save_folder, save_suffix = 'welch', format = format)

                        chp.model_overview_detailed(animals[sig][noise][sim]['protocol'], animals[sig][noise][sim]['best_models_fixed'], save_folder = save_folder, format = format)
                        chp.model_overview(animals[sig][noise][sim]['protocol'], animals[sig][noise][sim]['best_models'], save_folder = save_folder, format = format)
                        chp.model_over_signal(animals[sig][noise][sim]['protocol'], animals[sig][noise][sim]['best_models'], position = 'head', mv_avg_window = 1, save_folder = save_folder, format = format)

                        chp.time_serie(animals[sig][noise][sim]['protocol'], title = 'TIME SERIES', x_label = 'TIME (DAYS)', y_label = 'AMPLITUDE', save_folder = save_folder, format = format)

                        print('Signal: ', sig, 'Noise: ', noise, 'Simulation: ', sim)

Signal:  triangle Noise:  0 Simulation:  0_triangle_0
Signal:  triangle Noise:  0 Simulation:  1_triangle_0
Signal:  triangle Noise:  0 Simulation:  2_triangle_0
Signal:  triangle Noise:  0 Simulation:  3_triangle_0
Signal:  triangle Noise:  0 Simulation:  4_triangle_0
Signal:  triangle Noise:  0 Simulation:  5_triangle_0
Signal:  triangle Noise:  0 Simulation:  6_triangle_0
Signal:  triangle Noise:  0 Simulation:  7_triangle_0
Signal:  triangle Noise:  0 Simulation:  8_triangle_0
Signal:  triangle Noise:  0 Simulation:  9_triangle_0
Signal:  triangle Noise:  0.1 Simulation:  0_triangle_0.1
Signal:  triangle Noise:  0.1 Simulation:  1_triangle_0.1
Signal:  triangle Noise:  0.1 Simulation:  2_triangle_0.1
Signal:  triangle Noise:  0.1 Simulation:  3_triangle_0.1
Signal:  triangle Noise:  0.1 Simulation:  4_triangle_0.1
Signal:  triangle Noise:  0.1 Simulation:  5_triangle_0.1
Signal:  triangle Noise:  0.1 Simulation:  6_triangle_0.1
Signal:  triangle Noise:  0.1 Simulation:  7_triangle_

### RUN THE MODELING FOR SPECIFIED DAY WINDOWS

In [ ]:
dict = {'day_window': 1, 
        'step': 0.01, 
        'start_time': 22, 
        'end_time': 26, 
        'n_components': [1]}

best_models = []

for count, animal in enumerate(animals):
    init = time.time()

    save_folder = animals[animal]['save_folder']

    best_models_per_day, best_models_fixed_file = chrt.fit_cosinor_per_day(animals[animal]['protocol'], dict = dict, plot = True, save_folder = save_folder)    
    animals[animal]['best_models_per_day'] = best_models
    animals[animal]['best_models_file_per_day'] = best_models_file

    end = time.time() - init

    print("Cosinor fitted to " + animal.replace('_', ' ') + " and results saved!")
    print("Time elapsed: " + "{:.2f}".format(end) + " seconds")